In [1]:
import os
from boxsdk import OAuth2, Client
import re
import warnings

from datetime import datetime, date
from uuid import uuid4
from dateutil.tz import tzlocal
from ast import literal_eval

import numpy as np
import pandas as pd
import warnings
import librosa
import pickle

from pynwb import NWBHDF5IO, NWBFile, TimeSeries
from pynwb.file import Subject
from pynwb.epoch import TimeIntervals
from pynwb.image import ImageSeries
from ndx_manoli_meta import AssayMetadata

from nwb_utils import get_date_from_block
from box_utils import *
from behavior_error_checks import *

In [3]:
# ---- Get login credentials from environment variables -----

# - note this needs to be adjusted for each authorized user... ask Nerissa about getting and setting this information
# - developer access tokens expire quickly, so this probably needs to be set at startup and perhaps even while working
client_id = os.environ.get('box_client_id')
client_secret = os.environ.get('box_client_secret')
access_token = 'ARNBR6v1pQ5WqFQREa7jLYAV6ThmHHWi'


# ----- Open a connection to the Box server -----

auth = OAuth2(
    client_id=client_id, # put these in environmental variables
    client_secret=client_secret,
    access_token=access_token,
)
client = Client(auth)

# ----- Work on directory structure -----

# get the list of items in the whole Scn2a folder
items = client.folder(folder_id='196168550606').get_items()

# list contents
for item in items:
    print(f'{item.type.capitalize()} {item.id} is named "{item.name}"')

Folder 238576018931 is named "Aggregated_Events"
Folder 202955487033 is named "April2023_ShortCoHab_Females"
Folder 226890392440 is named "BorisFiles"
Folder 196172227913 is named "Female Intros"
Folder 196173015061 is named "Female PPTs"
Folder 196170422974 is named "Female RI"
Folder 196169230497 is named "Female SepReunion"
Folder 196171174766 is named "Female TMs"
Folder 214178119262 is named "June2023_ControlBehaviors"
Folder 196166676922 is named "Male Intros"
Folder 196172624162 is named "Male PPTs"
Folder 196171675763 is named "Male RI"
Folder 196172896956 is named "Male SepReunion"
Folder 196172739371 is named "Male TMs"
Folder 248394520156 is named "Naive_Choice"
Folder 229587753245 is named "Oct2023_JuvenileBehavior"
File 1503021327997 is named "NC11_2024-02-14 11-43-05.mp4"
File 1177298175085 is named "Scn2aX1Nov22_Key.xlsx"


In [2]:
# ----- Set up user parameters
directory_keyword = 'SepReunion' # how the assay directories are labeled
metadata_file = 'metadata_reunions.csv' # where the initial metadata is stored
boris_keyword = 'SR'
score_path = os.path.join('Scn2a_X1Behavior_Nov22','Aggregated_Events')
assayregex = 'S|s'
# colors for plotting
fillcols = [[160, 146, 95],[245, 201, 39],[89, 91, 125],[63, 78, 245]]
linecols = [[96, 87, 57],[147, 120, 23],[53, 54, 75],[33, 41, 131]]

In [5]:
# ----- Figure out which directories have the assay of interest

projectDir = '196168550606'
rootpath = ['Scn2a_X1Behavior_Nov22']

usedirs = []
items = client.folder(folder_id=projectDir).get_items() # top level folder

for item in items:
    iid = item.id
    inm = item.name
    if inm.find(directory_keyword)>-1:
        usedirs.append(iid)

print(usedirs)

# --- collect all the paths and filenames

allpaths = []

for usedir in usedirs: 
    
    # add starting folder to curr_path
    udnm = client.folder(folder_id=usedir).get(fields=['name']).name
    curr_path = rootpath + [udnm]
    
    # search for files
    thesepaths = visit_all_dirs_files(usedir,[],curr_path,client)

    # consolidate
    allpaths += thesepaths
    
print(allpaths)

# ----- Get a list of just file names to hunt through -----
justfiles = []
for pth in allpaths:
    justfiles.append(os.path.basename(pth))
print(justfiles)

['196169230497', '196172896956']
['Scn2a_X1Behavior_Nov22\\Female SepReunion\\01092023\\DoNotScore_Nov22_Pair13_SR_2023-01-09 13-42-51.mp4', 'Scn2a_X1Behavior_Nov22\\Female SepReunion\\01092023\\DoNotScore_Nov22_Pair14_SR_2023-01-09 13-43-07.mp4', 'Scn2a_X1Behavior_Nov22\\Female SepReunion\\01092023\\Nov22_Pair15_SR_2023-01-09 13-44-53.mp4', 'Scn2a_X1Behavior_Nov22\\Female SepReunion\\01092023\\Nov22_Pair16_SR_2023-01-09 13-45-43.mp4', 'Scn2a_X1Behavior_Nov22\\Female SepReunion\\01302023\\Nov22_Pair21_SR_2023-01-30 11-27-39.mp4', 'Scn2a_X1Behavior_Nov22\\Female SepReunion\\01302023\\Nov22_Pair22_SR_2023-01-30 11-28-32.mp4', 'Scn2a_X1Behavior_Nov22\\Female SepReunion\\01302023\\Nov22_Pair23_SR_2023-01-30 11-28-56.mp4', 'Scn2a_X1Behavior_Nov22\\Female SepReunion\\02131023\\Nov22_Pair28_SR_2023-02-13 11-29-07.mp4', 'Scn2a_X1Behavior_Nov22\\Female SepReunion\\02131023\\Nov22_Pair29_SR_2023-02-13 11-29-10.mp4', 'Scn2a_X1Behavior_Nov22\\Female SepReunion\\02131023\\Nov22_Pair30_SR_2023-02-13

In [6]:
# ----- Use metadata table to find expected files and add those columns to the table -----

# load up metadata
meta = pd.read_csv(metadata_file)

# generate a regex of possible matches for filename
fileIndex = []
for tag in meta.PairTag:
    match = find_in_list(justfiles,tag,assayregex)
    if len(match)==1:
        fileIndex.append(match[0])
    elif len(match)==0:
        print(f'No match for: {tag}')
    else:
        print(f'Found extra matches for: {tag}')
        for idx in match:
            print(justfiles[idx])
        fileIndex.append(match[0])
        
# use fileIndex to look up file names and paths
metafiles = []
metapaths = []
for i,tag in enumerate(meta.PairTag):
    metafiles.append(justfiles[fileIndex[i]])
    metapaths.append('\\'+os.path.dirname(allpaths[fileIndex[i]]))

# add files and paths to the metadata table
meta.VideoFile = metafiles
meta.VideoPath = metapaths

In [7]:
# ----- Get BORIS scored aggregated events file names -----
aggfiles = []
items = client.folder(folder_id='238576018931').get_items()

for item in items:
    inm = item.name
    if inm.find(boris_keyword)>-1:
        aggfiles.append(inm)
        
print(aggfiles)
print(len(aggfiles))

['NK_011823_Nov22_Pair1_SR_WIN_20221120_13_40_22_Pro.csv', 'NK_Male_Nov22_Pair17_SR_2023-01-11 13-52-19.csv', 'NK_Male_Nov22_Pair18_SR_2023-01-11 13-52-21.csv', 'NK_Male_Nov22_Pair19_SR_2023-01-11 13-52-20.csv', 'NK_Male_Nov22_Pair24_SR_2023-02-01 15-07-38.csv', 'NK_Male_Nov22_Pair25_SR_2023-02-01 15-07-39 (imported at 2023-06-08 12_24_48).csv', 'NK_Male_Nov22_Pair26_SR_2023-02-01 15-07-41.csv', 'NK_Male_Nov22_Pair27_SR_WIN_20230201_15_07_57_Pro.csv', 'NK_Male_Nov22_Pair32_SR_2023-02-15 13-55-51.csv', 'NK_Male_Nov22_Pair33_SR_2023-02-15 13-55-53.csv', 'NK_Male_Nov22_Pair34_SR_2023-02-15 13-55-54.csv', 'NK_Male_Nov22_Pair37_SR_2023-03-08 14-25-10.csv', 'NK_Male_Nov22_Pair38_SR_2023-03-08 14-25-11.csv', 'NK_Male_Nov22_Pair39_SR_2023-03-22 13-15-58.csv', 'NK_Male_Nov22_Pair40_SR_2023-03-22 13-16-56.csv', 'NK_Male_Nov22_Pair41_SR_2023-03-22 13-17-46.csv', 'NK_Male_Nov22_Pair43_SR_2023-03-29 12-55-36.csv', 'NK_Male_Nov22_Pair44_SR_2023-04-12 14-26-48.csv', 'NK_Male_Nov22_Pair46_SR_2023-04-1

In [11]:
# ----- Add Boris csvs and colors to metadata table

# match pair tags
ordered_agg = []
for tag in meta.PairTag:
    for fname in aggfiles:
        if f'{tag}_' in fname:
            ordered_agg.append(fname)

meta.ScoreFile = ordered_agg

# add path
score_path_arr = [score_path]*len(aggfiles)
meta.ScorePath = score_path_arr

# ----- Add colors to meta file so they go in the NWB metadata -----

# meta.FocalColor
allcols = []
for i, ptag in enumerate(meta.PairTag):
    if meta.FocalSex[i]=='F':
        if meta.FocalGT[i]=='WT':
            tcol = [fillcols[0],linecols[0]]
        else:
            tcol = [fillcols[1],linecols[1]]
    else:
        if meta.FocalGT[i]=='WT':
            tcol = [fillcols[2],linecols[2]]
        else:
            tcol = [fillcols[3],linecols[3]]
    allcols.append(tcol)
    
meta.FocalColor=allcols

# ----- Fix bad date formatting -----
# dx = meta.pop('Unnamed: 0')

newrecdates = []
for i, olddate in enumerate(meta.RecDate):
    dsplit = olddate.split('/')
    yr = dsplit[2]
    mn = dsplit[0]
    dy = dsplit[1]
    if len(mn)==1:
        mn = '0'+mn
    if len(dy)==1:
        dy = '0'+dy
    newrecdates.append(yr+mn+dy)
    
meta.RecDate=newrecdates

# --- Check on metadata before writing it ---
meta.head()

,PairTag,AssayType,RecDate,RecTime,VideoFile,VideoPath,ScoreFile,ScorePath,FemaleID,FemaleGT,...,Timeline,Ethogram,RanBy,ScoredBy,FullTimeline,FocalColor,StrangerID,StrangerGT,PPTlane,PartnerChamber
0,Nov22_Pair1,reunion,20221120,14:40,Nov22_Pair1_SR_WIN_20221120_13_40_22_Pro.mp4,\Scn2a_X1Behavior_Nov22\Female SepReunion\1120...,NK_011823_Nov22_Pair1_SR_WIN_20221120_13_40_22...,Scn2a_X1Behavior_Nov22\Aggregated_Events,B8002,Het,...,NaN,NaN,Gina Williams,Nanditha Krishnan,NaN,"[[245, 201, 39], [147, 120, 23]]",NaN,NaN,NaN,NaN
1,Nov22_Pair15,reunion,20230109,14:51,Nov22_Pair15_SR_2023-01-09 13-44-53.mp4,\Scn2a_X1Behavior_Nov22\Female SepReunion\0109...,NK_Nov22_Pair15_SR_2023-01-09 13-44-53.csv,Scn2a_X1Behavior_Nov22\Aggregated_Events,B6614,Het,...,NaN,NaN,Gina Williams,Nanditha Krishnan,NaN,"[[245, 201, 39], [147, 120, 23]]",NaN,NaN,NaN,NaN
2,Nov22_Pair16,reunion,20230109,14:51,Nov22_Pair16_SR_2023-01-09 13-45-43.mp4,\Scn2a_X1Behavior_Nov22\Female SepReunion\0109...,NK_Nov22_Pair16_SR_2023-01-09 13-45-43.csv,Scn2a_X1Behavior_Nov22\Aggregated_Events,B6615,WT,...,NaN,NaN,Gina Williams,Nanditha Krishnan,NaN,"[[160, 146, 95], [96, 87, 57]]",NaN,NaN,NaN,NaN
3,Nov22_Pair17,reunion,20230111,15:02,Nov22_Pair17_SR_2023-01-11 13-52-19.mp4,\Scn2a_X1Behavior_Nov22\Male SepReunion\01112023,NK_Male_Nov22_Pair17_SR_2023-01-11 13-52-19.csv,Scn2a_X1Behavior_Nov22\Aggregated_Events,NaN,NaN,...,NaN,NaN,Gina Williams,Nanditha Krishnan,NaN,"[[63, 78, 245], [33, 41, 131]]",NaN,NaN,NaN,NaN
4,Nov22_Pair18,reunion,20230111,15:02,Nov22_Pair18_SR_2023-01-11 13-52-21.mp4,\Scn2a_X1Behavior_Nov22\Male SepReunion\01112023,NK_Male_Nov22_Pair18_SR_2023-01-11 13-52-21.csv,Scn2a_X1Behavior_Nov22\Aggregated_Events,NaN,NaN,...,NaN,NaN,Gina Williams,Nanditha Krishnan,NaN,"[[89, 91, 125], [53, 54, 75]]",NaN,NaN,NaN,NaN


In [12]:
# --- write metadata table
meta.to_csv(metadata_file,index=False)

In [4]:
# ----- Do data check on aggregated files before writing NWB files -----

meta = pd.read_csv(metadata_file)
writeToDisk = False

# keep track of overloaded files
multifile_log = []

# keep track of annotation overlaps
ol_files = []
ol_behav = []
ol_start = []
ol_end = []

# keep track of annotation gaps
g_files = []
g_b1 = []
g_b2 = []
g_end = []
g_start = []

for i, ptag in enumerate(meta.PairTag):
    
    # get assay duration
    duration = float(meta.AssayDuration[i])
    
    scoretab = pd.read_csv(os.path.join('..',meta.ScorePath[i],meta.ScoreFile[i])) # load up csv of annotations

    scoretab.rename(columns={'Start (s)':'start'}, inplace=True)
    scoretab.rename(columns={'Stop (s)':'end'}, inplace=True)
    scoretab.rename(columns={'Duration (s)':'duration'}, inplace=True)
    scoretab.rename(columns={'Behavior type':'behavior_type'}, inplace=True)

    # --- Check for problems in the annotation file

    # check for extraneous media files
    if len(np.unique(scoretab['Media file']))>1:
        warnings.warn(f'Scored csv {meta.ScoreFile[i]} contains events for multiple media files.')
        multifile_log.append(meta.ScoreFile[i])
        for fp in np.unique(scoretab['Media file']):
            print(fp)

    else:       

        # check for total event duration violations
        totdur = np.sum(scoretab.duration)
        if totdur>1.05*duration:
            warnings.warn(f'Total events duration of {meta.ScoreFile[i]} exceeds the assay duration.')
        elif totdur<0.95*duration:
            warnings.warn(f'Total events duration of {meta.ScoreFile[i]} does not meet the assay duration.')

        # check for overlapping events and check for unscored gaps between events
        # keep track of annotation overlaps
        ols,ole,olb,gs,ge,gb1,gb2 = check_for_event_interactions(scoretab,True)

        # log overlaps
        for j,ol in enumerate(ols):
            ol_files.append(meta.ScoreFile[i])
            ol_behav.append(olb[j])
            ol_start.append(ol)
            ol_end.append(ole[j])

        # log gaps
        for j, gp in enumerate(gs):
            g_files.append(meta.ScoreFile[i])
            g_b1.append(gb1[j])
            g_b2.append(gb2[j])
            g_end.append(ge[j])
            g_start.append(gs[j])

# --- Write annotation violation logs for review
ol_log = {'file':ol_files,'behavior':ol_behav,'start_time':ol_start,'end_time':ol_end}
ol_tab = pd.DataFrame(data=ol_log)
# ol_tab.to_csv(f'overlap_log_{meta.AssayType[i]}.csv',index=False)

gap_durs = []
for i,st in enumerate(g_start):
    gap_durs.append(st-g_end[i])
gap_log = {'file':g_files,'behavior1':g_b1,'behavior1_end':g_end,'behavior2':g_b2,'behavior2_start':g_start,
           'gap_duration':gap_durs}
g_tab = pd.DataFrame(data=gap_log)
# g_tab.to_csv(f'gap_log_{meta.AssayType[i]}.csv',index=False)

if writeToDisk:
    ol_tab.to_csv(f'overlap_log_{meta.AssayType[i]}.csv',index=False)
    g_tab.to_csv(f'gap_log_{meta.AssayType[i]}.csv',index=False)